<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Logistic Regression with Keras</H1></u></center>

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
from keras.datasets import mnist

## Preprocessing the data:

In [ ]:
# the data, shuffled and split between train and test sets 
(X_train_, y_train), (X_test_, y_test) = mnist.load_data()

In [ ]:
input_dim = 784 #28*28 
X_train = X_train_.reshape(60000, input_dim) 
X_test = X_test_.reshape(10000, input_dim) 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

In [ ]:
from keras.utils import np_utils
n_classes = len(set(y_train))
Y_train = np_utils.to_categorical(y_train, n_classes) 
Y_test = np_utils.to_categorical(y_test, n_classes)

## Creating the model:

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

In [ ]:
output_dim = n_classes = 10 
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 
batch_size = 128 
n_epoch = 20

## Compile the model:

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

## Train the model:

In [ ]:
#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
model.fit(X_train, Y_train, batch_size=batch_size, epochs=n_epoch,verbose=1, validation_data=(X_test, Y_test)) 

## Evaluate the model:

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

## CallBacks:

In [ ]:
import keras

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
history = LossHistory()

In [ ]:
model.fit(X_train, Y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, Y_test), callbacks=[history])

In [ ]:
print(history.losses)

## Save the model and weights:

In [ ]:
model.save_weights('../data/mnist_lr_weights.h5', overwrite=True)

In [ ]:
model.save('../data/mnist_lr.h5')

In [ ]:
from keras.models import load_model

In [ ]:
loaded_model = load_model('../data/mnist_lr.h5')

In [ ]:
preds = loaded_model.predict(X_test)

In [ ]:
preds[0]

## Printing numbers and predictions:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
n = 10
plt.figure(figsize=(15,15))
for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(X_test_[i], cmap='gray')
    plt.title("Label: {}\nPredicted: {}".format(y_test[i], np.argmax(preds[i])))
    plt.axis('off')
plt.show()

## Reference:

https://keras.io/callbacks/

https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model